In [ ]:
import scanpy as sc
from metadimm import MetaDIMM
import numpy as np
import pandas as pd
from CITEsort_rna.BTreeTraversal import BTreeTraversal
from CITEsort_rna.Visualize import plot_keymarker,visualize_tree

In [ ]:
id='1_5'
adata = sc.read_h5ad('../simulator_out/RNA_simulator_'+id+'.h5ad')

label = pd.read_csv('../simulator_out/droplets_composition_'+id+'.csv')
label.index = label.index.astype(str)
adata.obs['label'] = label['cell_type']

f = open('../output/'+id+'_rna/tree.pickle','rb')
# f = open('CITEsort_out/SeuratV3_main_ct_smooth2/tree.pickle','rb')
tree = pickle.load(f)
f.close()
traversal = BTreeTraversal(tree)
md = MetaDIMM()
adata = adata[tree.indices,:]
# adata = md.filter(adata)
adata = md.preprocess(adata, normalize=True, log1p=True, hvg=True, scale=True)

pred_rna = pd.read_csv('../output/'+id+'_rna/leaf_labels.csv',index_col=0)
pred_rna.index  = pred_rna.index.astype(str)
adata.obs['pred_rna'] = pred_rna['Label']
sc.tl.rank_genes_groups(adata, groupby='pred_rna', method='t-test')

In [ ]:
idnew = '2_1'
adata_new = sc.read_h5ad('../simulator_out/RNA_simulator_'+idnew+'.h5ad')
adata_npp = md.preprocess(adata, normalize=True, log1p=True, hvg=False, scale=True)

In [ ]:
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

queue = [] 
newdict = dict()
dedict = dict()
queue.append(tree) 
newdict[0] = adata_npp.obs_names
ind = 0
pred_new = pd.Series(index=adata_new.obs_names,dtype='object')

while(len(queue) > 0): 
    node = queue.pop(0) 
    if node.left is not None:
        queue.append(node.left)
        queue.append(node.right) 

        adata_sub = adata[node.indices,:]
        adata_sub.obs['node_split'] = pd.Series(dtype='object')
        adata_sub.obs['node_split'].loc[node.left.indices] = str(0)
        adata_sub.obs['node_split'].loc[node.right.indices] = str(1)
        sc.tl.rank_genes_groups(adata_sub, groupby='node_split', method='t-test')
        DE_genes = pd.DataFrame(adata_sub.uns['rank_genes_groups']['names'][:1000])
        genes = list(set(list(DE_genes.loc[:,'0'])+list(DE_genes.loc[:,'1'])))

        clf = LinearDiscriminantAnalysis()
        clf.fit(adata_sub[:,genes].X, adata_sub.obs['node_split'])
        # clf.predict()

        # adata_sub = adata_npp[newdict[ind],genes]
        # pred = pd.Series(clf.predict(adata_sub),index=adata_sub.obs_names)
        # maxind = max(newdict.keys())
        # newdict[maxind+1] = adata_sub[pred=='0'].obs_names
        # newdict[maxind+2] = adata_sub[pred=='1'].obs_names
        # dedict[maxind+1] = DE_genes.loc[:,'0'].values
        # dedict[maxind+2] = DE_genes.loc[:,'1'].values

        # adata_sub = adata_new[newdict[ind],list(set(list(DE_genes.loc[:,'0'])+list(DE_genes.loc[:,'1'])))]
        # adata_sub =  md.preprocess(adata_sub, normalize=True, log1p=True, hvg=False, scale=True)
        # sc.tl.pca(adata_sub,n_comps=20)
        # gm = GaussianMixture(n_components=2).fit(adata_sub.obsm['X_pca'])
        # pred = pd.Series(gm.predict(adata_sub.obsm['X_pca']),index=adata_sub.obs_names)
        # tmp = [sum(sum(adata_sub[pred==0,DE_genes.loc[:,'0']].X))/sum(pred==0),sum(sum(adata_sub[pred==1,DE_genes.loc[:,'0']].X))/sum(pred==1)]
        # maxind = max(newdict.keys())
        # newdict[maxind+1] = adata_sub[pred==np.argmax(tmp),:].obs_names
        # newdict[maxind+2] = adata_sub[pred==(1-np.argmax(tmp)),:].obs_names
        # dedict[maxind+1] = DE_genes.loc[:,str(np.argmax(tmp))].values
        # dedict[maxind+2] = DE_genes.loc[:,str(1-np.argmax(tmp))].values
    else:
        pred_new.loc[newdict[ind]] = str(ind)+ '_leaf'
    ind = ind+1
    break

In [ ]:
idnew = '2_1'
label = pd.read_csv('../simulator_out/droplets_composition_'+idnew+'.csv')
label.index = label.index.astype(str)
adata_npp.obs['label'] = label['cell_type']
sc.pl.umap(adata_npp,color='label')